In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime,timedelta

In [2]:
pd.set_option("max_columns", 100)

In [3]:
def fecha_MDA(value):
    date=[]
    for i in value:
#         print(i)
        try:
            date.append(datetime.strptime(str(i), "%Y-%d-%m %H:%M:%S").strftime('%d-%m-%Y'))
        except:
            date.append(i.replace('/','-'))
#         print(type(date[-1]),date[-1])
    return date

In [4]:
def Upload_Dicc(name):
    with open(name,'r') as archivo:
        separador,dicc = '\t',{}
        for linea in archivo:
            key, value = linea.split(separador)
            dicc[key.strip()] = value.strip()
    return dicc

# 

In [5]:
Data=pd.read_excel('Base De Datos Humedad PLC.xlsx')
# Data['Date']=fecha_MDA(Data['Date'])
# Data['Time']=Data['Time'].astype(str)
# Data.insert(0,'Date_Time',pd.to_datetime(Data['Date']+'-'+Data['Time'],format='%m-%d-%Y-%H:%M:%S'))
# Data=Data.drop(columns=['Date','Time'])
Data.head()

,Date,Time,Millitm,{[SUPREMA]MIC_202.Val_PV},{[SUPREMA]MIC_202.Val_SP},{[SUPREMA]MIC_203.Val_PV},{[SUPREMA]MIC_203.Val_SP},{[SUPREMA]MIC_302.Val_PV},{[SUPREMA]MIC_302.Val_SP},{[SUPREMA]MIC_304.Val_PV},{[SUPREMA]MIC_304.Val_SP},{[SUPREMA]TE_121.VALOR},{[SUPREMA]TE_302.VALOR},{[SUPREMA]TE_305A.VALOR},{[SUPREMA]TE_401.VALOR},{[SUPREMA]TIC_121.OSet_SP},{[SUPREMA]TIC_201.Val_PV},{[SUPREMA]TIC_201.Val_SP},{[SUPREMA]TIC_202.Val_PV},{[SUPREMA]TIC_202.Val_SP},{[SUPREMA]TIC_203.Val_PV},{[SUPREMA]TIC_203.Val_SP},{[SUPREMA]TIC_302.OSet_SP},{[SUPREMA]TIC_305.Val_PV},{[SUPREMA]TIC_305.Val_SP},{[SUPREMA]TIC_305A.OSet_SP},{[SUPREMA]TIC_401.OSet_SP}
0,2022-02-02 00:00:00,23:30:59,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,79.988220,71.718750,32.242035,31,64.366730,64.5,73.771606,74.5,76.527435,77.5,77.0,77.503128,77.199997,77.531250,31
1,2022-02-02 00:00:00,23:31:59,125,90.931252,89.699997,83.087502,86.0,69.606255,69.699997,77.006248,77.0,30,78.658386,72.187500,33.027115,31,64.736267,64.5,75.854492,74.5,77.440704,77.5,77.0,77.681252,77.199997,77.653130,31
2,2022-02-02 00:00:00,23:32:59,126,90.706253,89.699997,82.700005,86.0,69.731255,69.699997,77.012505,77.0,30,79.811981,72.590630,33.491760,31,61.988647,64.5,72.441772,74.5,76.831848,77.5,77.0,77.868752,77.199997,77.906250,31
3,2022-02-02 00:00:00,23:33:59,127,91.206253,89.699997,82.849998,86.0,68.962502,69.699997,77.087502,77.0,31,80.020264,72.975006,33.667999,31,64.366730,64.5,74.524658,74.5,77.248444,77.5,77.0,78.028130,77.199997,78.046875,31
4,2022-02-02 00:00:00,23:34:59,128,88.568748,89.699997,81.681252,86.0,69.724998,69.699997,77.106255,77.0,31,77.408661,72.337502,33.924362,31,63.659729,64.5,75.373840,74.5,77.440704,77.5,77.0,77.709381,77.199997,77.681252,31


In [6]:
Data.shape

(41667, 27)

In [7]:
Dicc_est=Upload_Dicc('Diccionario Estaciones.txt')
Dicc_deltat=Upload_Dicc('Diccionario Deltat.txt')

In [8]:
Data_filter=(Data
             .rename(columns=Dicc_est)
             .filter(regex=(r'\A\w{2}\W?\w{3}|\A\w{4,7}'))
             .drop(index=0)
             .reset_index(drop=True)
            )
Data_filter.head()

,Date,Time,Millitm,ME-202,ME-203,ME-302,ME-304,TE-302,TE-201,TE-202,TE-203
0,2022-02-02 00:00:00,23:31:59,125,90.931252,83.087502,69.606255,77.006248,78.658386,64.736267,75.854492,77.440704
1,2022-02-02 00:00:00,23:32:59,126,90.706253,82.700005,69.731255,77.012505,79.811981,61.988647,72.441772,76.831848
2,2022-02-02 00:00:00,23:33:59,127,91.206253,82.849998,68.962502,77.087502,80.020264,64.366730,74.524658,77.248444
3,2022-02-02 00:00:00,23:34:59,128,88.568748,81.681252,69.724998,77.106255,77.408661,63.659729,75.373840,77.440704
4,2022-02-02 00:00:00,23:35:59,129,89.193748,83.256248,69.337502,76.856255,80.693207,62.727783,72.425751,76.303131


In [9]:
Data_filter.shape

(41666, 11)

In [11]:
time_ini,time_end,TE_201,ME_202,TE_202,ME_203,TE_203,TE_302,ME_302,ME_304=[],[],[],[],[],[],[],[],[],[]
rango=[TE_201,ME_202,TE_202,ME_203,TE_203,TE_302,ME_302,ME_304]
apuntador=0
# for i in range(1000):
for i in Data_filter.index:
    try:
        time_ini.append(Data_filter.loc[i,'Date'])
        time_end.append(Data_filter.loc[i,'Time'])
        for j,k in enumerate(Dicc_deltat.keys()):
            apuntador=i+int(Dicc_deltat[k])
#             print(j,k,apuntador)
            rango[j].append(Data_filter.loc[apuntador,k])
    except:
        break;
time_ini.pop(-1)
time_end.pop(-1)

datetime.time(6, 35, 43)

In [12]:
for i in range(len(rango)-1):
    rango[i].pop(-1)
#     print(len(rango[i]))

In [16]:
Data_Final=pd.DataFrame({'Date':time_ini,'Time':time_end,'TE-201':TE_201,'ME-202':ME_202,'TE-202':TE_202,'ME-203':ME_203,'TE-203':TE_203,
                         'TE-302':TE_302,'ME-302':ME_302,'ME-304':ME_304})
Data_Final=Data_Final.set_index(['Date','Time'])

In [17]:
Data_Final.tail()

TE-201     ME-202     TE-202     ME-203     TE-203  \
Date       Time                                                              
03/31/2022 06:30:43  63.691864  92.262505  74.460571  84.006248  77.200378   
           06:31:43  63.000946  93.118752  72.842316  83.275002  78.273865   
           06:32:43  64.366730  88.337502  74.845093  83.131248  76.944000   
           06:33:43  62.422485  92.587502  73.835693  82.806252  78.658386   
           06:34:43  64.013214  89.856255  72.986511  82.568748  76.992065   

                        TE-302     ME-302     ME-304  
Date       Time                                       
03/31/2022 06:30:43  78.402039  68.425003  76.781250  
           06:31:43  80.661163  68.500000  76.775002  
           06:32:43  77.937378  68.618752  76.787498  
           06:33:43  80.933533  68.318748  76.793755  
           06:34:43  78.850647  68.693748  76.800003

In [15]:
Data_Final.shape

(41463, 10)

In [19]:
# Data_Final.to_excel('sfsdsdd.xlsx')

In [21]:
def preprocess(BD):
    data = list(tf.constant(BD, dtype="float32"))  
    return tf.data.Dataset.from_tensor_slices(data).map(lambda seq: (tf.reshape(seq[:-1], (-1, 1)), seq[-1])).batch(64, drop_remainder=True)

In [26]:
dataset1 = preprocess( Data_Final[['TE-201', 'ME-202', 'TE-202', 'ME-203', 'TE-203', 'TE-302','ME-302','ME-304']].values)

In [27]:
dataset1

<BatchDataset element_spec=(TensorSpec(shape=(64, 7, 1), dtype=tf.float32, name=None), TensorSpec(shape=(64,), dtype=tf.float32, name=None))>

In [31]:
for X, y in dataset1.take(1):
    print("X")
    print(X)
    print("y")
    print(y)

X
tf.Tensor(
[[[64.73627 ]
  [90.81875 ]
  [75.43793 ]
  [82.20625 ]
  [77.40866 ]
  [77.32855 ]
  [69.71875 ]]

 [[61.988647]
  [90.65625 ]
  [72.553925]
  [81.95625 ]
  [76.96002 ]
  [79.68381 ]
  [69.168755]]

 [[64.36673 ]
  [90.387505]
  [75.24567 ]
  [81.81875 ]
  [76.86389 ]
  [77.05618 ]
  [69.33125 ]]

 [[63.65973 ]
  [88.78125 ]
  [74.46057 ]
  [82.8375  ]
  [77.60092 ]
  [79.58768 ]
  [69.05    ]]

 [[62.727783]
  [88.78125 ]
  [72.5379  ]
  [82.86875 ]
  [76.39926 ]
  [77.536835]
  [68.90625 ]]

 [[64.89697 ]
  [90.525   ]
  [75.30975 ]
  [83.44375 ]
  [77.84125 ]
  [80.6131  ]
  [69.4875  ]]

 [[64.591675]
  [88.5375  ]
  [72.98651 ]
  [81.5625  ]
  [76.09485 ]
  [78.86667 ]
  [69.3125  ]]

 [[62.56711 ]
  [89.2125  ]
  [74.44455 ]
  [83.106255]
  [77.56888 ]
  [78.369995]
  [69.725   ]]

 [[64.51132 ]
  [89.7125  ]
  [75.966644]
  [82.856255]
  [76.11087 ]
  [80.2446  ]
  [69.24375 ]]

 [[62.743835]
  [89.981255]
  [72.07327 ]
  [82.3     ]
  [77.31253 ]
  [77.632965]
  [